# CoursewareHubのパラメータ変更

---

構築したCoursewareHubのパラメータを変更します。

## 概要


### グループ名

変更対象となるCoursewareHubを構築したときに指定したansibleのグループ名を指定します。

ansibleのグループ名を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上のセルの出力結果を参考にしてグループ名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

指定されたグループ名が前提条件を満たしていることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

CoursewareHubのサービスが実行されていることを確認します。

In [ ]:
cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
!ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"

group_varsから現在の設定値を読み込みます。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

## パラメータの更新

### Single-user Server

ユーザの利用環境となるSingle-user Serverに関するパラメータを指定します。

#### ユーザインターフェース

デフォルトの表示をClassic Notebook InterfaceとJupyterLabのどちらにするかを指定します。

現在の設定値を確認します。次のセルを実行すると、どちらのユーザインターフェースが指定されているかが表示されます。

In [ ]:
default_url = gvars.get("jupyterhub_singleuser_default_url", "/lab")
if  default_url == "/tree":
    print("Classic Notebook Interface")
elif default_url == "/lab":
    print("JupyterLab")
else:
    raise RuntimeError(default_url)

現在の設定値から変更する場合は次のセルのどちらかの行のコメントを外して実行してください。

In [ ]:
# jupyterhub_singleuser_default_url = '/tree'   # Classic Notebook Interface
# jupyterhub_singleuser_default_url = '/lab'     # JupyterLab

#### リソース制限の設定

single-user Serverに割り当てるリソースの制限に関する設定を行います。

##### リソース制限の設定方法

CoursewareHubではユーザの役割に応じて利用方法が異なることがあります。例えば講師権限ユーザーは採点を行う場合などに複数のNotebookを同時に開くことがあります。そのような利用を行う場合、一般の受講者ユーザよりも多くのリソースが必要となります。また講師権限を与えないが、採点の補助を行う役割をもつTA(Teaching Assistant)のユーザーがいる場合があります。それらのユーザには一般の受講者ユーザーとは異なるリソース制限設定を行うことが考えられます。

これらのことに対応するためにCoursewareHubでは権限もしくは所属グループに応じたリソース制限の設定を行うことができます。

リソース制限の設定はYAMLファイルで行います。YAMLファイルの記述例を以下に示します。

```yaml
groups:
    student:
        mem_limit: 1G
        cpu_limit: 1.0
        priority: 10
    teaching-assistant:
        mem_limit: 2G
        cpu_limit: 2.0
        mem_guarantee: 1G
        cpu_guarantee: 0.5
        priority: 1
admin:
    mem_limit: 5G
default:
    mem_limit: 1G
    cpu_limit: 2.0
    mem_guarantee: 1G
    cpu_guarantee: 0.5
```

上の例では `student`グループ、`teaching-assistant`グループ、講師権限ユーザ(`admin`)、それ以外のユーザ（デフォルト設定）についてリソース制限の設定を行っています。複数のグループに所属するユーザについては、グループの`priority`が小さいほうのグループの設定が優先されます。上記の例では`student`グループ、`teaching-assistant`グループの両方に属するユーザは `priority`が`1`となっている`teaching-assistant`グループの設定が優先されます。

コンテナに対するリソース制限設定として以下の４つの属性を指定することができます。

* [mem_guarantee](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.Spawner.mem_guarantee)
    - コンテナの使用が保証されるメモリサイズの下限
* [mem_limit](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.Spawner.mem_limit)
    - コンテナが使用可能なメモリのサイズの上限
* [cpu_guarantee](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.LocalProcessSpawner.cpu_guarantee)
    - コンテナの使用が保証される CPU 使用率の下限
* [cpu_limit](https://jupyterhub.readthedocs.io/en/stable/api/spawner.html#jupyterhub.spawner.LocalProcessSpawner.cpu_limit)
    - コンテナが使用可能な CPU 使用率の上限

##### リソース制限の設定を行うYAMLファイルを作成する

現在、設定されている値を確認します。

In [ ]:
!cat {gvars["rsc_yml"]}

設定されている値を変更する場合はこの後のセルの実行を続けてください。現在の設定のままで良い場合は、次の節までスキップしてください。

リソースファイルを作成するディレクトリを作成します。

In [ ]:
from tempfile import mkdtemp
from pathlib import Path
rsc_pdir = Path('./rsc').resolve()
rsc_pdir.mkdir(exist_ok=True)
rsc_dir = Path(mkdtemp(dir=rsc_pdir))

YAMLファイルを作成します。

リソース制限の設定に対応する内容に次のセルを変更してください。その後、セルを実行するとYAMLファイルが作成されます。

In [ ]:
%%writefile {rsc_dir}/resource.yaml
default:
    mem_limit: 1G
    mem_guarantee: 1G
    cpu_limit: 1.0

admin:
    mem_limit: 4G
    mem_guarantee: 1G
    cpu_limit: 1.0

groups:
    staff:
        mem_limit: 4G
        mem_guarantee: 1G
        cpu_limit: 1.0
        priority: 1


YAMLファイルの記述内容が妥当であるかをチェックします。

次のセルが正常に実行できることを確認してください。実行結果がエラーとなった場合はYAMLファイルの記述内容に問題があります。上のセルを unfreeze して記述内容を修正して再実行してください。

In [ ]:
import jsonschema
import json
import yaml

!ansible -c local {target_hub} -m get_url -a \
    'url=https://raw.githubusercontent.com/NII-cloud-operation/CoursewareHub-LC_platform/master/jupyterhub/resources-schema.json \
    dest={rsc_dir}/resources-schema.json'
with (rsc_dir / 'resources-schema.json').open() as f:
    resources_config_schema = json.load(f)

with (rsc_dir / 'resource.yaml').open() as f:
    resources_config = yaml.load(f, Loader=yaml.SafeLoader)
    jsonschema.validate(resources_config, resources_config_schema)

print(json.dumps(resources_config, indent=2))

現在の設定との差分を確認します。

In [ ]:
!diff -u {gvars["rsc_yml"]} {rsc_dir}/resource.yaml || true

### JupyterHub

JupyterHubコンテナに設定するパラメータを指定します。

#### Cullingの設定

アイドル状態のSingle-user Serverを自動的に停止する機能に関するパラメータを指定します。

ここで指定できるパラメータを以下に示します。

|項目 | 説明 | デフォルト値|
|:---|:---|--:|
|cull_server              | アイドル状態のNotebookサーバーの停止を有効／無効化 (yes/no/0/1)| no |
|cull_server_idle_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間(秒)     | 600 |
|cull_server_max_age      | アイドル状態でなくてもNotebookサーバーを停止するまでの時間(秒)     | 0 |
|cull_server_every       | Notebookのアイドル状態のタイムアウトのチェック間隔(秒)             | 0 |



現在の設定値を確認します。

In [ ]:
print(f"""cull_server={gvars.get('cull_server', 'no')}
cull_server_idle_timeout={gvars.get('cull_server_idle_timeout', 600)}
cull_server_max_age={gvars.get('cull_server_max_age', 0)}
cull_server_every={gvars.get('cull_server_every', 0)}""")

現在の設定値から変更する場合は、対応する以下のセルに値を指定して実行してください。設定値を変更する必要がないセルは実行せずにスキップしてください。

In [ ]:
# (例)
# cull_server = "yes"       # 自動停止を有効にする
# cull_server = "no"        # 自動停止を無効にする

cull_server = 

In [ ]:
# (例)
# cull_server_idle_timeout = 10 * 60               # １０分間
# cull_server_idle_timeout = 12 * 60 * 60      # 12時間

cull_server_idle_timeout = 

In [ ]:
# (例)
# cull_server_max_age = 0
# cull_server_max_age = 24 * 60 * 60      # 24時間

cull_server_max_age = 

In [ ]:
# (例)
# cull_server_every = 0
# cull_server_every = 60

cull_server_every = 

#### JupyterHubのSpawnerの設定

Single-user Serverの起動に関するパラメータを指定します。

ここで指定できるパラメータを以下に示します。

|項目 | 説明 | デフォルト値 |
|:---|:---|---:|
|concurrent_spawn_limit   | ユーザーコンテナ同時起動処理数                                 | 20 |
|spawner_http_timeout     | ユーザーNotebookサービス 起動タイムアウト時間(秒)              | 120 |
|spawner_start_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間(秒)  | 300 |

現在の設定値を確認します。

In [ ]:
print(f"""concurrent_spawn_limit={gvars.get('concurrent_spawn_limit', 20)}
spawner_http_timeout={gvars.get('spawner_http_timeout', 120)}
spawner_start_timeout={gvars.get('spawner_start_timeout', 300)}""")

現在の設定値から変更する場合は、対応する以下のセルに値を指定して実行してください。設定値を変更する必要がないセルは実行せずにスキップしてください。

In [ ]:
# (例)
# concurrent_spawn_limit = 20

concurrent_spawn_limit = 

In [ ]:
# (例)
# spawner_http_timeout = 120

spawner_http_timeout = 

In [ ]:
# (例)
# spawner_start_timeout = 300

spawner_start_timeout = 

#### ロゴ

JupyterHubのロゴを更新します。変更しない場合は次の節までスキップしてください。

現在の設定値を確認します。次のセルの実行結果に何も表示されない場合はデフォルトのロゴとなっています。

In [ ]:
print(gvars.get("logo_file", ""))

このNotebookを実行している環境にロゴファイルを配置してそのパスを次のセルで指定してください。

In [ ]:
# (例)
# logo_file = 'jupyterhub-logo.png'

logo_file = 

#### カスタマイズ

JupyterHubのカスタマイズ用設定ファイルを更新します。変更しない場合は次の節までスキップしてください。

現在の設定値を確認します。次のセルの実行結果に何も表示されない場合は設定ファイルのカスタマイズは行われていません。

In [ ]:
for x in gvars.get("jupyterhub_config_list", []):
    print(x)

用意した設定ファイルの名前を次のセルのリストに指定してください。既に指定しているファイルも有効にするのであれば、既存のファイル名も次のセルのリストに指定する必要があります。またファイル名の末尾は`.py`であるものだけが有効になります。

In [ ]:
jupyterhub_config_list = [
    # (例)
    #  'share_jupyterhub_config.py',
]

設定ファイルの記述方法については以下のページなどを参照してください。

* [JupyterHub - Configuration Reference](https://jupyterhub.readthedocs.io/en/stable/reference/config-reference.html)

### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

if 'rsc_dir' in vars() and (Path(rsc_dir) / 'resource.yaml').exists():
    rsc_yml = str((Path(rsc_dir) / 'resource.yaml').resolve())

if 'logo_file' in vars():
    logo_file = str(Path(logo_file).resolve())
if 'jupyterhub_config_list' in vars():
    jupyterhub_config_list = [str(Path(x).resolve()) for x in jupyterhub_config_list]

jupyterhub_param_names = [
    'rsc_yml',
    'cull_server',
    'cull_server_idle_timeout',
    'cull_server_max_age',
    'cull_server_every',
    'concurrent_spawn_limit',
    'spawner_http_timeout',
    'spawner_start_timeout',
    'logo_file',
    'jupyterhub_config_list',
    'intermediate_certfile',
    'jupyterhub_singleuser_default_url',
]

for name in jupyterhub_param_names:
    if name in vars():
        gvars[name] = vars()[name]

gvars_path.rename(gvars_path.parent / f".{gvars_path.stem}.old")
with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat group_vars/{ugroup_name}

## コンテナの更新

### JupyterHubコンテナに関するファイルの配置

JupyterHubコンテナを実行するために必要となるファイルを配置します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/var/jupyterhub/resource.yaml|リソース割り当て設定ファイル|
|/var/jupyterhub/logo.png|ロゴファイル|
|/etc/jupyterhub/jupyterhub_config.d/|カスタマイズ設定ファイルディレクトリ|


まず、実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-jupyterhub.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-jupyterhub.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_cfg_dir}}}}'

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_var_dir}}}}'

### docker-compose.yml の配置

複数のコンテナに関する設定をまとめて扱うために `docker-compose.yml` を利用します。ここまでに指定されたパラメータに応じた`docker-compose.yml`を生成しmanagerノードの`/opt/coursewarehub/docker-compose.yml`に配置します。

まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/deploy-docker-compose.yml || true

実際に`docker-compose.yml`の配置を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/deploy-docker-compose.yml

配置したファイルを確認します。異なるパラメータで何度か配置を行った場合は、以前に配置したファイルがバックアップファイルとして残されています。

In [ ]:
!ansible {target_hub} -b -a 'tree -F {{{{compose_dir}}}}'

### コンテナの起動

コンテナを起動します。

起動中のコンテナを一度停止します。

In [ ]:
!ansible {target_hub} -a 'docker stack rm coursewarehub'

改めてコンテナを起動し直します。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{compose_dir}}}} \
    docker stack deploy -c docker-compose.yml coursewarehub'

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker service ls'

全てのコンテナが起動するまで待ち合わせを行います。

> 次のセルの実行結果がエラーとなる場合は、その後のセルを実行してコンテナの実行状況やログを確認してください。

In [ ]:
import time
import sys


cmd = "docker service ls --format '{{.Name}} {{.Replicas}}' | grep '1/1' | grep coursewarehub_ | wc -l"
cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
for retry in range(18):
    time.sleep(10)
    try:
        !ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"
        break
    except:
        print('retry', file=sys.stderr)
else:
    !ansible {target_hub} -a 'docker service ls'
    raise RuntimeError("起動処理が完了しませんでした。")

`postgres` コンテナのログを表示してみます。

In [ ]:
!ansible {target_hub} -a 'docker service logs coursewarehub_postgres'

`jupyterhub` コンテナのログを表示してみます。

> PostgreSQLに接続できないなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs coursewarehub_jupyterhub'

`auth-proxy` コンテナのログを表示してみます。

> 証明書設定に誤りがあるなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs coursewarehub_auth-proxy'

## CoursewareHubにアクセスする

パラメータを更新したCoursewareHub環境にアクセスして正しく動作していることを確認してください。

次のセルを実行するとCoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{gvars["master_fqdn"]}')